In [191]:
import pandas as pd
import mysql.connector
import sqlalchemy

In [196]:
cnx = mysql.connector.connect(user='root', password='password', 
                              host='127.0.0.1', database='otus_db', port='3306')

In [197]:
cur = cnx.cursor()

### Создадим таблицу products_info
#### product_id - идентификатор продукта
#### product_name - наименование продукта 
#### properties - свойства продукта 
#### description - описание продукта

In [240]:
cur.execute('''
            DROP TABLE IF EXISTS products_info;
            ''')

cur.execute('''
            CREATE TABLE products_info (
                product_id BIGINT UNSIGNED NOT NULL AUTO_INCREMENT PRIMARY KEY,
                product_name VARCHAR(255) NOT NULL,
                properties VARCHAR(255) NOT NULL,
                description VARCHAR(255) NOT NULL)
                ;
            ''')

 #### Наполним таблицу случайными данными размером 1000 строк 

In [241]:
cur.execute('''
INSERT INTO products_info (product_name, properties, description)
SELECT 
  CASE 
    WHEN RAND() < 0.25 THEN 'testy'
    WHEN RAND() < 0.5 THEN 'very testy food'
    WHEN RAND() < 0.75 THEN 'something cool'
    ELSE 'dont eat it'
  END AS product_name,
  CASE 
    WHEN RAND() < 0.25 THEN 'hot food'
    WHEN RAND() < 0.5 THEN 'drinks'
    WHEN RAND() < 0.75 THEN 'vegetables'
    ELSE 'fruits'
  END AS properties,
  CASE 
    WHEN RAND() < 0.33 THEN 'Made in Russia'
    WHEN RAND() < 0.66 THEN 'Made in USA'
    ELSE 'Made in China'
  END AS description
FROM 
  (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t1
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t2
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t3
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t4
   CROSS JOIN (SELECT 1 AS n UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4) t5
   LIMIT 1000;
            ''')

query = f'''
          SELECT
          *
         from products_info
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,properties,description
0,1,something cool,drinks,Made in USA
1,2,very testy food,fruits,Made in USA
2,3,testy,hot food,Made in USA
3,4,very testy food,hot food,Made in Russia
4,5,testy,drinks,Made in China
...,...,...,...,...
995,996,testy,drinks,Made in Russia
996,997,something cool,hot food,Made in USA
997,998,something cool,drinks,Made in Russia
998,999,something cool,vegetables,Made in Russia


In [251]:
cur.execute('''
            set profiling = 1;
            ''')

query = f'''
        select* from products_info
        where product_name  like '%food%'
        and properties like '%hot%'
        and description like '%Russia%'
        '''
df = pd.read_sql_query(query, cnx)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,properties,description
0,4,very testy food,hot food,Made in Russia
1,27,very testy food,hot food,Made in Russia
2,109,very testy food,hot food,Made in Russia
3,111,very testy food,hot food,Made in Russia
4,126,very testy food,hot food,Made in Russia


In [252]:
query = f'''
            show profiles;
        '''
df = pd.read_sql_query(query, cnx)
df.tail(1)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Query_ID,Duration,Query
14,49,0.004788,select* from products_info\n where prod...


In [253]:
query = f'''
        EXPLAIN
        select* from products_info
        where product_name  like '%food%'
        and properties like '%hot%'
        and description like '%Russia%'
        '''
df = pd.read_sql_query(query, cnx)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products_info,None,ALL,None,None,None,None,1000,0.14,Using where


### Создадим полнотекстовый индекс на:
#### product_name - наименование продукта 
#### properties - свойства продукта 
#### description - описание продукта

In [245]:
cur.execute('''
            CREATE FULLTEXT index 
                fulltext_inx_product_name
                on products_info 
                (product_name)
                ;
            ''')

cur.execute('''
            CREATE FULLTEXT index 
                fulltext_inx_properties
                on products_info 
                (properties)
                ;
            ''')


cur.execute('''
            CREATE FULLTEXT index 
                fulltext_inx_description
                on products_info 
                (description)
                ;
            ''')

### Посмотреть индексы таблицы

In [246]:
query = f'''
        SHOW INDEX from products_info
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,products_info,0,PRIMARY,1,product_id,A,1000,None,None,,BTREE,,,YES,None
1,products_info,1,fulltext_inx_product_name,1,product_name,None,1000,None,None,,FULLTEXT,,,YES,None
2,products_info,1,fulltext_inx_properties,1,properties,None,1000,None,None,,FULLTEXT,,,YES,None
3,products_info,1,fulltext_inx_description,1,description,None,1000,None,None,,FULLTEXT,,,YES,None


In [247]:
query = f'''
        EXPLAIN
        select* from products_info
        where match (product_name)  against ('food')
        and match (properties)  against ('hot')
        and match (description)  against ('Russia')
        
        '''
df = pd.read_sql_query(query, cnx)
df

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,select_type,table,partitions,type,possible_keys,key,key_len,ref,rows,filtered,Extra
0,1,SIMPLE,products_info,None,fulltext,"fulltext_inx_product_name,fulltext_inx_propert...",fulltext_inx_product_name,0,const,1,5.0,Using where; Ft_hints: sorted


In [260]:
query = f'''
        select* from products_info
        where match (product_name)  against ('food')
        and match (properties)  against ('hot')
        and match (description)  against ('Russia')
        
        '''
df = pd.read_sql_query(query, cnx)
df.head()

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,product_id,product_name,properties,description
0,4,very testy food,hot food,Made in Russia
1,27,very testy food,hot food,Made in Russia
2,109,very testy food,hot food,Made in Russia
3,111,very testy food,hot food,Made in Russia
4,126,very testy food,hot food,Made in Russia


In [261]:
query = f'''
            show profiles;
        '''
df = pd.read_sql_query(query, cnx)
df.tail(1)

/Users/denis/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Query_ID,Duration,Query
14,54,0.003588,select* from products_info\n where matc...


In [262]:
cur.close()
cnx.close()